In [42]:
import pickle
import numpy as np
import torch
import seaborn as sns
import sys
# Imports from other modules and packages in the project
sys.path.append('../')

from src.helpers import *


## Load data

In [43]:
name = 'Chewie'
date = '1007'
fold = 4
target_variable = 'vel'

In [44]:
#@title Helper functions for plotting (run this cell!)
sns.set_context("notebook")

# initialize a color palette for plotting
palette = sns.xkcd_palette(["windows blue",
                            "red",
                            "medium green",
                            "dusty purple",
                            "orange",
                            "amber",
                            "clay",
                            "pink",
                            "greyish"])

## Load pre-processed data

In [45]:
data_path = '../Data/Processed_Data/Tidy_'+name+'_'+date+'.pkl'

with open(data_path, 'rb') as file:
    tidy_df = pickle.load(file)

## 1 - Force Adaptation Dataset

In [46]:
# Create filtered DataFrames
baseline_df = tidy_df.loc[tidy_df['epoch'] == 'BL']
force_df = tidy_df.loc[tidy_df['epoch'] == 'AD']
wo_df = tidy_df.loc[tidy_df['epoch'] == 'WO']

# Function to split IDs into 3 equal sets
def split_ids(df):
    unique_ids = df.id.unique()
    set_length = len(unique_ids) // 3
    return [unique_ids[i * set_length:(i + 1) * set_length] for i in range(3)]

# Split IDs for each epoch
ids_base = split_ids(baseline_df)
ids_force = split_ids(force_df)
ids_wo = split_ids(wo_df)

# Function to create dataset dictionary entries
def create_dataset_entry(df, ids, name_prefix):
    return {
        f'{name_prefix}{i+1}': get_dataset(df.loc[df.id.isin(ids[i])], fold, target_variable='vel', no_outliers=False, force_data=True)[:6]
        for i in range(3)
    }

# Create datasets dictionary
datasets_adaptation = {}
datasets_adaptation.update(create_dataset_entry(baseline_df, ids_base, 'Baseline'))
datasets_adaptation.update(create_dataset_entry(force_df, ids_force, 'Force'))
datasets_adaptation.update(create_dataset_entry(wo_df, ids_wo, 'Washout'))

Train trials 36
Test trials  11
Val trials 9
We are testing the optimization method on fold  4
Train trials 36
Test trials  11
Val trials 9
We are testing the optimization method on fold  4
Train trials 36
Test trials  11
Val trials 9
We are testing the optimization method on fold  4
Train trials 43
Test trials  13
Val trials 11
We are testing the optimization method on fold  4
Train trials 43
Test trials  13
Val trials 11
We are testing the optimization method on fold  4
Train trials 43
Test trials  13
Val trials 11
We are testing the optimization method on fold  4
Train trials 45
Test trials  14
Val trials 11
We are testing the optimization method on fold  4
Train trials 45
Test trials  14
Val trials 11
We are testing the optimization method on fold  4
Train trials 45
Test trials  14
Val trials 11
We are testing the optimization method on fold  4


In [47]:
dataset_names = list(datasets_adaptation.keys())
random.shuffle(dataset_names)

In [48]:
dataset_names

['Force3',
 'Washout1',
 'Baseline2',
 'Washout2',
 'Baseline1',
 'Washout3',
 'Force1',
 'Force2',
 'Baseline3']

In [51]:
order_adaptation_v3 = ['Baseline1',
                       'Baseline2',
                       'Force1',
                       'Force2',
                       'Washout1',
                       'Washout2',
                       'Baseline3',
                        'Force3',
                        'Washout3',]

In [52]:
# Reorder dictionary
ordered_datasets_adaptation = {key: datasets_adaptation[key] for key in order_adaptation_v3}


In [53]:
data_dir = './Data'
path_to_save_data = os.path.join(data_dir, 'Real_Data_'+'Adaptation_v2'+'.pkl')

# Pickle the data and save it to file
with open(path_to_save_data, 'wb') as handle:
    pickle.dump(ordered_datasets_adaptation, handle, protocol=4)

## 2 - Stimulation Dataset

In [24]:
data_path = '../Data/Processed_Data/Tidy_Sansa_0504.pkl'

with open(data_path, 'rb') as file:
    df = pickle.load(file)

In [25]:
baseline_df_s = df.loc[df.type == 'BASELINE'].reset_index()
tonic_df_s = df.loc[df.type == 'TONIC'].reset_index()
stim_df_s = df.loc[df.type == 'BC-REACH'].reset_index()

In [26]:
print('Total number of trials for baseline :', baseline_df_s.id.nunique())
print('Total number of trials for tonic stimulation :', tonic_df_s.id.nunique())
print('Total number of trials for triggered stimulation :', stim_df_s.id.nunique())

Total number of trials for baseline : 29
Total number of trials for tonic stimulation : 35
Total number of trials for triggered stimulation : 69


In [27]:
fold = 4

In [28]:
# Function to create dataset dictionary entries
def create_dataset_entry_stim(df, ids, name_prefix, num_sets):
    return {
        f'{name_prefix}{i+1}': get_dataset(df.loc[df.id.isin(ids[i])], fold, target_variable='target_vel', no_outliers=False, force_data=False)[:6]
        for i in range(num_sets)
    }


ids_base_s = [baseline_df_s.id.unique()]
ids_tonic_s = [tonic_df_s.id.unique()]
shuffled_ids = stim_df_s.id.unique()
random.shuffle(shuffled_ids)
ids_stim_s = [shuffled_ids[:45], shuffled_ids[45:]]

# Create datasets dictionary
datasets_stimulation = {}
datasets_stimulation.update(create_dataset_entry_stim(baseline_df_s, ids_base_s, 'Baseline', 1))
datasets_stimulation.update(create_dataset_entry_stim(tonic_df_s, ids_tonic_s, 'Tonic', 1))
datasets_stimulation.update(create_dataset_entry_stim(stim_df_s, ids_stim_s, 'Triggered', 2))

Train trials 18
Test trials  6
Val trials 5
We are testing the optimization method on fold  4
Train trials 22
Test trials  7
Val trials 6
We are testing the optimization method on fold  4
Train trials 29
Test trials  9
Val trials 7
We are testing the optimization method on fold  4
Train trials 15
Test trials  5
Val trials 4
We are testing the optimization method on fold  4


In [29]:
# Desired key order
desired_order = ['Baseline1', 'Triggered1', 'Tonic1','Triggered2']
desired_order_2 = ['Triggered1', 'Baseline1', 'Triggered2', 'Tonic1']
# Reorder dictionary
ordered_datasets = {key: datasets_stimulation[key] for key in desired_order}
ordered_datasets_2 = {key: datasets_stimulation[key] for key in desired_order_2}

In [30]:
data_dir = './Data'
path_to_save_data = os.path.join(data_dir, 'Real_Data_'+'Stimulation_v3'+'.pkl')

# Pickle the data and save it to file
with open(path_to_save_data, 'wb') as handle:
    pickle.dump(ordered_datasets_2, handle, protocol=4)